In [ ]:
import numpy as np
import matplotlib
matplotlib.use('PDF')
from pylab import *
dir = '/Users/cslage/Research/LSST/code/imSim/imSim/data/tree_ring_data/'

In [ ]:
figure()
plotcounter = 1
filenames = ['itl-017-770nm_save_points.txt', 'itl-098-770nm_save_points.txt', 'itl-206-770nm_save_points.txt', 'itl-283-770nm_save_points.txt', 'itl-350-770nm_save_points.txt', 'e2v-260-70V_save_points.txt']
numfiles = len(filenames)
tree_ring_data = []
f_k = []
fftfreq = []
subplots_adjust(hspace=1.5, wspace=0.5)
for n, filename in enumerate(filenames):
    name = filename[0:7]
    tree_ring_data.append(np.loadtxt(dir+filename))
    bad_indices = []
    for i,[xval,fval] in enumerate(tree_ring_data[n]):
        if np.isnan(fval) or xval < 1000.0 or xval > 5800.0:
            bad_indices.append(i)

    tree_ring_data[n] = np.delete(tree_ring_data[n], bad_indices, axis=0)        
    deltax = tree_ring_data[n][100,0] - tree_ring_data[n][99,0]
    #print filename, deltax
    f_k.append(np.fft.fft(tree_ring_data[n][-500:-1,1]))
    fftfreq.append(np.fft.fftfreq(len(f_k[n]), d = deltax))
    #print x[0], f[0]
    subplot(numfiles,2,plotcounter)
    title("Measured Area Deviation(%%)-%s"%name, fontsize=6)
    plot(tree_ring_data[n][:,0],tree_ring_data[n][:,1], color='blue')
    yticks([-1.0,0.0,1.0])
    xticks([2000.0,4000.0,6000.0])
    subplot(numfiles,2,plotcounter+1)
    title("Measured Area Deviation(%%)-%s"%name, fontsize=6)
    plot(tree_ring_data[n][-250:-1,0],tree_ring_data[n][-250:-1,1], color='blue')
    if n == 0:
        yticks([-1.0,0.0,1.0])
    elif n == 1:
        yticks([-0.5,0.0,0.5])
    else:
        yticks([-0.1,0.0,0.1])
    xticks([5000.0,5400.0,5800.0])
    B0 = 0.02
    B = 2.0E-16
    #f2 = B0 + B * x**4
    A = 1.2E-3
    L = 1000.0
    #f3 = B0 + A * (exp(x/L) - 1.0)
    #plot(x,f2,color='red')
    #plot(x,f3,color='green')
    #ylim(-0.1, 0.1)
    plotcounter += 2
    #legend(loc = 'lower center', fontsize=6)
#savefig(dir+'Tree_Ring_Data_13Feb18.pdf')
#show()
close()

In [ ]:
meank1 = 60.0
sigmak1 = 10.0
meank2 = 35.0
sigmak2 = 10.0

figure()
subplots_adjust(hspace=1.5, wspace=0.5)
startfreq = 10
stopfreq = 110
numpoints = stopfreq-startfreq
for n in range(numfiles):

    name = filenames[n][0:7]
    L = []
    for i in range(startfreq,stopfreq):
        L.append(1/fftfreq[n][i]) # This converts the k values into pixel frequencies
    
    cfreqs = f_k[n].real[startfreq:stopfreq]
    sfreqs = f_k[n].imag[startfreq:stopfreq]
    power = (cfreqs**2 + sfreqs**2)
    subplot(numfiles,1,n+1)
    title("Spectral Power-%s"%name, fontsize=6)
    plot(L, power)
    if n < 2:
        yticks([0.0, 400.0])
        ylim(0,400)
    elif n<3:
        yticks([0.0, 4.0])
        ylim(0,4)
    else:
        yticks([0.0, 2.0])
        ylim(0,2)
    L = array(L)
    model_power = power.max() * 0.75 * exp(-(L-meank1)*(L-meank1) /(2.0 * sigmak1*sigmak1))
    model_power += power.max() * 0.25 * exp(-(L-meank2)*(L-meank2) /(2.0 * sigmak2*sigmak2))
    plot(L, model_power, color='red', ls = '--')
xlabel("Period(Pixels)")
savefig(dir+'Tree_Ring_Frequencies_30Apr18.pdf')
#show()
close()


In [ ]:
figure()
B0 = 2E-3
np.random.seed(1241)
numfreqs1 = 15
meank1 = 60.0
sigmak1 = 10.0
numfreqs2 = 5
meank2 = 35.0
sigmak2 = 10.0
subplots_adjust(hspace=1.5, wspace=0.5)
numtrials = numfiles

for trial in range(numtrials):
    if rand() < 0.4:
        B = np.random.uniform(4.0E-17, 8.0E-17, 1)[0]
    else:
        B = np.random.uniform(4.0E-18, 8.0E-18, 1)[0]
    print trial, B
    cfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    #cfreqs = np.random.uniform(40.0, 75.0, numfreqs)
    cphases = np.random.uniform(0.0, 2*pi, numfreqs)
    sfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    #sfreqs = np.random.uniform(40.0, 75.0, numfreqs)
    sphases = np.random.uniform(0.0, 2*pi, numfreqs)
    ftrial = zeros([tree_ring_data[0].shape[0]])
    centroid_trial = zeros([tree_ring_data[0].shape[0]])
    for i,xval in enumerate(tree_ring_data[0][:,0]):
        for j,fval in enumerate(cfreqs):
            ftrial[i] += cos(2*pi*(xval/fval)+cphases[j])
            centroid_trial[i] += sin(2*pi*(xval/fval)+cphases[j]) * fval / (2.0*pi)
        for j,fval in enumerate(sfreqs):
            ftrial[i] += sin(2*pi*(xval/fval)+sphases[j])
            centroid_trial[i] += -cos(2*pi*(xval/fval)+sphases[j]) * fval / (2.0*pi)

        ftrial[i] *= (B0 + B * xval**4)
        centroid_trial[i] *= (B0 + B * xval**4) * .01 # 0.01 factor is becaue data is in percent
    subplot(numtrials,3,3*trial+1)
    title("Area Deviation(%%) Sample %d"%trial, fontsize=6)
    plot(tree_ring_data[0][:,0],ftrial,color='blue')
    ylim(-1.0,1.0)
    xticks([2000.0,4000.0,6000.0])
    yticks([-1.0,0.0,1.0])
    subplot(numtrials,3,3*trial+2)
    title("Measured Area Deviation(%)", fontsize=6)
    plot(tree_ring_data[trial][:,0],tree_ring_data[trial][:,1], color='green')
    ylim(-1.0,1.0)
    xticks([2000.0,4000.0,6000.0])
    yticks([-1.0,0.0,1.0])
    subplot(numtrials,3,3*trial+3)
    title("Centroid Shift (Pixels) Sample %d"%trial, fontsize=6)
    plot(tree_ring_data[0][:,0],centroid_trial,color='red')
    ylim(-0.05, 0.05)
    xticks([2000.0,4000.0,6000.0])
    yticks([-0.05,0.0,0.05])
savefig(dir+'Tree_Ring_Trials_Full_13Feb18.pdf')
#show()
close()

In [ ]:
figure()
B0 = 2E-3
np.random.seed(1241)
numfreqs1 = 15
meank1 = 60.0
sigmak1 = 10.0
numfreqs2 = 5
meank2 = 35.0
sigmak2 = 10.0
subplots_adjust(hspace=1.5, wspace=0.5)
numtrials = numfiles

for trial in range(numtrials):
    if rand() < 0.4:
        B = np.random.uniform(4.0E-17, 8.0E-17, 1)[0]
    else:
        B = np.random.uniform(4.0E-18, 8.0E-18, 1)[0]
    print trial, B
    cfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    cphases = np.random.uniform(0.0, 2*pi, numfreqs)
    sfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    sphases = np.random.uniform(0.0, 2*pi, numfreqs)
    ftrial = zeros([tree_ring_data[0].shape[0]])
    centroid_trial = zeros([tree_ring_data[0].shape[0]])
    for i,xval in enumerate(tree_ring_data[0][:,0]):
        for j,fval in enumerate(cfreqs):
            ftrial[i] += cos(2*pi*(xval/fval)+cphases[j])
            centroid_trial[i] += sin(2*pi*(xval/fval)+cphases[j]) * fval / (2.0*pi)
        for j,fval in enumerate(sfreqs):
            ftrial[i] += sin(2*pi*(xval/fval)+sphases[j])
            centroid_trial[i] += -cos(2*pi*(xval/fval)+sphases[j]) * fval / (2.0*pi)

        ftrial[i] *= (B0 + B * xval**4)
        centroid_trial[i] *= (B0 + B * xval**4) * .01 # 0.01 factor is becaue data is in percent
    subplot(numtrials,3,3*trial+1)
    title("Area Deviation(%%) Sample %d"%trial, fontsize=6)
    plot(tree_ring_data[0][-250:-1,0],ftrial[-250:-1],color='blue')
    ylim(-1.0,1.0)
    xticks([5000.0,5400.0,5800.0])
    yticks([-1.0,0.0,1.0])
    subplot(numtrials,3,3*trial+2)
    title("Measured Area Deviation(%)", fontsize=6)
    plot(tree_ring_data[trial][-250:-1,0],tree_ring_data[trial][-250:-1,1], color='green')
    ylim(-1.0,1.0)
    xticks([5000.0,5400.0,5800.0])
    yticks([-1.0,0.0,1.0])
    subplot(numtrials,3,3*trial+3)
    title("Centroid Shift (Pixels) Sample %d"%trial, fontsize=6)
    plot(tree_ring_data[0][-250:-1,0],centroid_trial[-250:-1],color='red')
    ylim(-0.05, 0.05)
    xticks([5000.0,5400.0,5800.0])
    yticks([-0.05,0.0,0.05])
savefig(dir+'Tree_Ring_Trials_Mag_13Feb18.pdf')
#show()
close()

In [ ]:
Ltest = 139
ftest = cos(2*pi*x/Ltest)

In [ ]:
f_k_test = np.fft.fft(ftest)
fftfreq = np.fft.fftfreq(len(f_k_test), d = deltax)
print fftfreq[0:20]

In [ ]:
plot(x,ftest)
show()

In [ ]:
cfreqs = f_k_test.real[0:250]
subplot(1,2,1)
title("Real Part")
plot(cfreqs)
sfreqs = f_k_test.imag[0:250]
subplot(1,2,2)
title("Imaginary Part")
plot(sfreqs)
show()

cfvals = []
cweights = []
for i,k  in enumerate(cfreqs):
    if abs(k) > 300.0 and k > freqs[i-1] and k > freqs[i+1] and i>2:
    #if abs(k) > 1.0 and i>2:
        print i,k, 1.0/fftfreq[i], Ltest
        cfvals.append(1.0/fftfreq[i])
        cweights.append(1.0)
sfvals = []
sweights = []
for i,k  in enumerate(sfreqs):
    if abs(k) > 300.0 and k>freqs[i-1] and k > freqs[i+1] and i>2:
    #if abs(k) > 1.0 and i>2:
        print i,k, 1.0/fftfreq[i], Ltest
        sfvals.append(1.0/fftfreq[i])
        sweights.append(1.0)



In [ ]:
subplots_adjust(hspace=1.0)
for trial in range(5):
    cphases = zeros([len(cfvals)])
    for j,fval in enumerate(cfvals):
        cphases[j] = rand()
    sphases = zeros([len(sfvals)])
    for j,fval in enumerate(sfvals):
        sphases[j] = rand()

    ftrial = zeros([len(x)])
    for i,xval in enumerate(x):
        for j,fval in enumerate(cfvals):
            ftrial[i] += cweights[j] * cos(2*pi*(xval/fval+cphases[j]))
        for j,fval in enumerate(sfvals):
            ftrial[i] += sweights[j] * sin(2*pi*(xval/fval+sphases[j]))
        ftrial[i] *= 1.0
    subplot(6,1,trial+1)
    plot(x,ftrial,color='blue')
    ylim(-2.0,2.0)

subplot(6,1,6)
plot(x,ftest,color='green')
ylim(-2.0,2.0)

show()


In [ ]:
print tree_ring_data[0][:,0].shape
# Check of smallness of slowly varying derivative
figure()
B0 = 2E-3
np.random.seed(1241)
numfreqs1 = 15
meank1 = 60.0
sigmak1 = 10.0
numfreqs2 = 5
meank2 = 35.0
sigmak2 = 10.0
numfreqs = numfreqs1 + numfreqs2
subplots_adjust(hspace=1.5, wspace=0.5)
numtrials = 1

for trial in range(numtrials):
    B = 8E-17
    cfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    cphases = np.random.uniform(0.0, 2*pi, numfreqs)
    sfreqs = np.concatenate((np.random.normal(meank1, sigmak1, numfreqs1), np.random.normal(meank2, sigmak2, numfreqs2)))
    sphases = np.random.uniform(0.0, 2*pi, numfreqs)
    ftrial = zeros([tree_ring_data[0].shape[0]])
    centroid_trial = zeros([tree_ring_data[0].shape[0]])
    small_term_trial = zeros([tree_ring_data[0].shape[0]])
    
    for i,xval in enumerate(tree_ring_data[0][:,0]):
        for j,fval in enumerate(cfreqs):
            centroid_trial[i] += sin(2*pi*(xval/fval)+cphases[j]) * fval / (2.0*pi)
            small_term_trial[i] += cos(2*pi*(xval/fval)+cphases[j])
        for j,fval in enumerate(sfreqs):
            centroid_trial[i] += -cos(2*pi*(xval/fval)+sphases[j]) * fval / (2.0*pi)
            small_term_trial[i] += sin(2*pi*(xval/fval)+sphases[j])

        small_term_trial[i] *= (4.0 * B * xval**3) * 0.01
        centroid_trial[i] *= (B0 + B * xval**4) * .01 # 0.01 factor is because data is in percent
    print centroid_trial.max(), small_term_trial.max()
    subplot(1,1,1)
    plot(tree_ring_data[0][:,0],centroid_trial,color='blue')
    plot(tree_ring_data[0][:,0],small_term_trial,color='red')
#savefig(dir+'Tree_Ring_Trials_Mag_13Feb18.pdf')
show()
close()